In [4]:
# REQUIREMENTS
! pip install ipyleaflet
! pip install bs4
! pip install requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 5.0 MB/s 


In [52]:
#Import libraries
from ipyleaflet import Map, basemaps, basemap_to_tiles, TileLayer
from bs4 import BeautifulSoup
import requests
import ast #Abstract syntax trees 


### Scrape all available URLS using Beautiful Soup


In [40]:
#This URL can be changed
url='https://mapcore-demo.org/current/flatmap/v2/'
req= requests.get(url)
content=req.text
url_dict = ast.literal_eval(content)

In [50]:
#JOB : There are fields with non IDs, len=6 eliminated this, think of a smarter way!
model_ids = [model_dict['id'] for model_dict in url_dict if len(model_dict) == 6]
model_name = [model_dict['name'] for model_dict in url_dict if len(model_dict) == 6]
  

### Get all layers for each model to be added sequentially to the map

In [75]:
model_layer_url_list = ['https://mapcore-demo.org/current/flatmap/v2/flatmap/'+id+'/layers'   for id, name in zip(model_ids, model_name)]

In [98]:
model_image_layers = []
for url in model_layer_url_list:
  req= requests.get(url)
  content=req.text
  layer_url_dict = ast.literal_eval(content)
  model_image_layers.append({layer_url_dict[0]['id'] : layer_url_dict[0]['image-layers']})



In [122]:
model_image_layers[1]

{'whole-rat': ['whole-rat_image',
  'whole-rat_details_vagus_image',
  'whole-rat_details_tissue-slide_image']}

### Create a map with all layers of a model added seqentially to a map

In [134]:
#Choose one of the models in tag (There are 11 in the main URL)
tag = 2
tile_urls = ['https://mapcore-demo.org/current/flatmap/v2/flatmap/' + model_ids[tag] + '/tiles/' + layer + '/{z}/{x}/{y}' for layer in model_image_layers[tag][model_name[tag]]]
tile_urls


['https://mapcore-demo.org/current/flatmap/v2/flatmap/1220ab6b172448ccf9dd8c4d748447248d3185b25123dd5b4700d33c04d80011/tiles/whole-pig_image/{z}/{x}/{y}']

In [135]:
#Create a base map object with the first tile
map = Map(
    basemap=TileLayer(
                    url=tile_urls[0], 
                    min_zoom=0),
    min_zoom=0,
    zoom=0,
    scroll_wheel_zoom=True,
    dragging=True,
    attribution_control=False,
    zoom_snap=False,
)

#Create tile objects for the model with each layer added
for tile_url in tile_urls[1:]:

  map.add(TileLayer(
    url=tile_url, 
    min_zoom=0))

In [136]:
map

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…